In [14]:
import pandas as pd
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
from dash.dependencies import Input, Output

# Load your CSV data into a DataFrame (replace 'your_file.csv' with the actual file path)
df = pd.read_csv('word_analysis.csv')

# Get unique company names for dropdown options
company_options = [{'label': company, 'value': company} for company in df['Company Name'].unique()]
company_options.append({'label': 'All', 'value': 'All'})  # Add "All" option

# Get the list of word buckets
bucket_columns = df.columns[4:]  # Columns containing word bucket counts

# Initialize the Dash app
app = dash.Dash(__name__)

# Define the layout of the app
app.layout = html.Div([
    dcc.Dropdown(id='company-dropdown', options=company_options, value='All', multi=False),
    dcc.Graph(id='sentiment-bar-chart')
])

# Define color mappings for sentiment labels
color_discrete_map = {
    'Negative': 'red',
    'Neutral': 'blue',
    'Positive': 'green'
}

# Callback to update the sentiment bar chart
@app.callback(
    Output('sentiment-bar-chart', 'figure'),
    [Input('company-dropdown', 'value')]
)
def update_sentiment_bar_chart(selected_company):
    if selected_company == 'All':
        filtered_df = df
    else:
        filtered_df = df[df['Company Name'] == selected_company]
    
    sentiment_scores = filtered_df.groupby('Sentiment Label')[bucket_columns].sum().T
    
    fig = px.bar(
        sentiment_scores,
        x=sentiment_scores.index,
        y=sentiment_scores.columns,
        title='Sentiment Scores by Word Buckets',
        labels={'x': 'Word Bucket', 'y': 'Sentiment Score'},
        category_orders={"x": bucket_columns},  # Maintain order of word buckets on x-axis
        color_discrete_map=color_discrete_map  # Specify color mapping for sentiment labels
    )
    
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)